# soh estimation experimentation of Teslavehicles


## Setup

In [ ]:
! mkdir -p data_cache

### Imports

In [ ]:
import plotly.express as px
from scipy import stats
import numpy as np
from core.stats_utils import *
from core.pandas_utils import *
from core.config import valid_soh_points
from core.stats_utils import filter_results_by_lines_bounds
from transform.fleet_info.main import fleet_info
from transform.processed_tss.ProcessedTimeSeries import ProcessedTimeSeries

### Data extraction

In [ ]:
tss = ProcessedTimeSeries("tesla", force_update=False)

In [ ]:
tss.columns

In [ ]:
tss = tss.query("owner == 'ayvens'")

In [ ]:
print(tss["vin"].nunique())

In [ ]:
# sanity_check(tss)

In [ ]:
fleet_info.query("make == 'tesla'")["range"].value_counts(dropna=False, sort=True, ascending=False)

## Time series

In [ ]:
most_common_vin = tss.groupby("vin").size().sort_values(ascending=False).idxmax()
most_common_vin
vin = "5YJ3E7EA4LF551854"
ts = tss.query(f"vin == '{vin}'")

In [ ]:
px.scatter(ts, x="date", y="odometer", title=f"{vin}")

In [ ]:
tss.head()

#### Rolling variance


In [ ]:
# # Calculation on the rolling variance 
# tss["rolling_variance"] = tss.groupby("vin")["battery_energy"].transform(lambda group: group.rolling(window=3).var())
# var = tss.dropna(subset=['rolling_variance'])


In [ ]:
# # Heat map 
# # Sélectionner les colonnes d'intérêt
# columns_of_interest = ['soc', 'battery_energy', 'rolling_variance']

# # Calculer la matrice de corrélation
# correlation_matrix = tss[columns_of_interest].corr()
# # Visualiser la matrice de corrélation avec une heatmap
# fig = px.imshow(correlation_matrix,
#                 labels=dict(x="Variables", y="Variables", color="Correlation"),
#                 x=correlation_matrix.columns,
#                 y=correlation_matrix.index,
#                 title="Heatmap de la Corrélation")

# # # Afficher le graphique
# fig.show()

## First filtering 

In [ ]:
# Deleting odometer = 0 
ts = ts.query("odometer != 0")
tss = tss.query("odometer != 0")

## Reducing depandicies to factors


In [ ]:

tss_soh = (
    tss
    .query("trimmed_in_charge")
        .groupby(["vin", "trimmed_in_charge_idx"])
        .agg(
            energy_added=pd.NamedAgg("charge_energy_added", series_start_end_diff),
            soc_diff=pd.NamedAgg("soc", series_start_end_diff),
            soc_start=pd.NamedAgg("soc", "first"),
            soc_end=pd.NamedAgg("soc", "last"),
            temp=pd.NamedAgg("inside_temp", "mean"),
            capacity=pd.NamedAgg("capacity", "first"),
            odometer=pd.NamedAgg("odometer", "first"),
            fast_charger_type=pd.NamedAgg("fast_charger_type", Series.mode),
            size=pd.NamedAgg("soc", "size"),
            model=pd.NamedAgg("model", "first"),
            version=pd.NamedAgg("version", "first"),
            date=pd.NamedAgg("date", "first"),
        )
        .reset_index(drop=False)
        .eval("soh = energy_added / (soc_diff / 100 * capacity)")
        .eval("model_version = model + version")
)
ts_soh = (
    ts
    .query("trimmed_in_charge")
        .groupby(["vin", "trimmed_in_charge_idx"])
        .agg(
            energy_added=pd.NamedAgg("charge_energy_added", series_start_end_diff),
            soc_diff=pd.NamedAgg("soc", series_start_end_diff),
            soc_start=pd.NamedAgg("soc", "first"),
            soc_end=pd.NamedAgg("soc", "last"),
            temp=pd.NamedAgg("inside_temp", "mean"),
            capacity=pd.NamedAgg("capacity", "first"),
            odometer=pd.NamedAgg("odometer", "first"),
            fast_charger_type=pd.NamedAgg("fast_charger_type", Series.mode),
            size=pd.NamedAgg("soc", "size"),
            model=pd.NamedAgg("model", "first"),
            version=pd.NamedAgg("version", "first"),
            date=pd.NamedAgg("date", "first"),
        )
        .reset_index(drop=False)
        .eval("soh = energy_added / (soc_diff / 100 * capacity)")
        .eval("model_version = model + version")
)

### Depandicies to soc -> We take only for soc_diff > 40 / No dependices to soc_start and end

In [ ]:
px.scatter(ts_soh,
           x="odometer",
           y="soh"  , 
           hover_data=["vin"],
           color="soc_diff")

In [ ]:
px.scatter(ts, 
           x="odometer",
           y="soc",
           hover_data=["vin"],
           color="soc_diff")

### Finding other depandicies


In [ ]:
px.scatter(tss_soh.query("soc_diff > 40"),
           x="odometer",
           y="soh",
           hover_data=["vin"],
           color="soc_diff")

##  Final SOH

### Estimation

## Visualization

In [ ]:
px.scatter(ts_soh.query("vin == '5YJ3E7EA5KF470232'"), x="soc_diff", y="soh")


In [ ]:
from transform.raw_tss.main import get_raw_tss
raw_tss = get_raw_tss("mercedes-benz", force_update=False)
raw_tss.columns

The resulting sohs follows the overall trend which makes a lot more sense than the previous results.  
We can assume that the informed default ranges in fleet info are wrong.

## Conclusion

Soh from estimated range seems promessing and could be used as our final resulsts to Ayvens.  
We would, however, need to improve the accuracy of the estimator.  